In [137]:
import pandas as pd
import csv
import numpy as np
import random

In [138]:
random.seed(2003)
np.random.seed(2003)

## Get data as dataframes

In [10]:
df = pd.read_csv("data/all_tweets.csv")
for i, row in df.iterrows():
    print(row['tweets'])
    print('=======================================================')

I’ve got to start taking better care of myself. Tomorrow I’ll walk to the liquor store.
Mom: did you get an A on your spanish test
Me: C
Mom: okay mr mexico 🤩
At least I can say I tried. I didn’t try, but I can say I did.
people who separate your hahahas into ha ha ha, whats going on there
Sometimes, when I’m on a date with my wife of sixteen years, I break awkward silences by saying first date things like “so, I’ve never dated a mom before”
Pizza rolls are shaped like little pillows because you’re ready for a nap after eating 50 or 60 of them.
Me in a huff: you just said disrobe down to my socks. You didn’t say anything about staying in the exam room
[struggling to get out of a hammock] come here and say that
If an astronaut goes really fast they’re a fastronaut
I’ve reached the part of my summer where I’m excited that I’ll soon be able to stop checking my kids’s heads for ticks and go back to just checking for lice
I toss and turn in bed all night like a beautiful rotisserie chicken.

In [22]:
df['id'] = np.random.choice(range(100000, 1000000), df.shape[0], replace=False)
df['id'] = df['id'].apply(lambda x: int(x))

print(df)

          id                                             tweets
0     773725  I’ve got to start taking better care of myself...
1     153477  Mom: did you get an A on your spanish test\nMe...
2     343353  At least I can say I tried. I didn’t try, but ...
3     656823  people who separate your hahahas into ha ha ha...
4     684187  Sometimes, when I’m on a date with my wife of ...
...      ...                                                ...
1349  454395  no one: \nnot a soul:\nmy daughter: if I ever ...
1350  847523  I love my sister and even when I struggled as ...
1351  190160  Called in, “Let’s make this simple. What excus...
1352  539637  I dreamed I won $10 million on a slot machine ...
1353  972219  Watching a movie on the plane? No thanks. Watc...

[1354 rows x 2 columns]


In [23]:
df.to_csv('data/all_tweets.csv', index=False)

In [21]:
df['tweets'][0]

'I’ve got to start taking better care of myself. Tomorrow I’ll walk to the liquor store.'

In [35]:
annotated = pd.read_csv("data/funny-annotations.csv")
annotated

,id,funny
0,656823,0
1,684187,1
2,509083,1
3,865864,0
4,413339,1
...,...,...
294,111406,0
295,933761,1
296,922725,0
297,684064,0


In [152]:
# get only tweets that are identified as funny
funny_df = df.merge(annotated, how='inner', on=['id']).query('funny == 1')
funny_df = funny_df.reset_index().drop(columns=['index'])
funny_df

,id,tweets,funny
0,684187,"Sometimes, when I’m on a date with my wife of ...",1
1,509083,Pizza rolls are shaped like little pillows bec...,1
2,413339,[struggling to get out of a hammock] come here...,1
3,880531,If an astronaut goes really fast they’re a fas...,1
4,973171,I toss and turn in bed all night like a beauti...,1
...,...,...,...
138,144630,if you sweat while you eat it should count as ...,1
139,328630,god!! did anyone stop by your house and drop o...,1
140,971535,If you breakdance you buy dance.,1
141,604980,"You can catch a lot of flies with honey, but y...",1


## Identify 'most important' word to mask

In [73]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/phuongdang/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/phuongdang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/phuongdang/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [76]:
nltk.download('tagsets_json')
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets_json to
[nltk_data]     /Users/phuongdang/nltk_data...
[nltk_data]   Unzipping help/tagsets_json.zip.


In [110]:
funny_df['tweets'][1]

'Sometimes, when I’m on a date with my wife of sixteen years, I break awkward silences by saying first date things like “so, I’ve never dated a mom before”'

In [111]:
sample_sentence = funny_df['tweets'][1]
tokens = nltk.word_tokenize(sample_sentence)
pos_tags = nltk.pos_tag(tokens)
print(pos_tags)

[('Sometimes', 'RB'), (',', ','), ('when', 'WRB'), ('I', 'PRP'), ('’', 'VBP'), ('m', 'NNS'), ('on', 'IN'), ('a', 'DT'), ('date', 'NN'), ('with', 'IN'), ('my', 'PRP$'), ('wife', 'NN'), ('of', 'IN'), ('sixteen', 'JJ'), ('years', 'NNS'), (',', ','), ('I', 'PRP'), ('break', 'VBP'), ('awkward', 'JJ'), ('silences', 'NNS'), ('by', 'IN'), ('saying', 'VBG'), ('first', 'JJ'), ('date', 'NN'), ('things', 'NNS'), ('like', 'IN'), ('“', 'NNP'), ('so', 'RB'), (',', ','), ('I', 'PRP'), ('’', 'VBP'), ('ve', 'JJ'), ('never', 'RB'), ('dated', 'VBD'), ('a', 'DT'), ('mom', 'NN'), ('before', 'IN'), ('”', 'NN')]


In [87]:
# determine word uniqueness, complexity(?) based on its appearance in the brown text corpus
from nltk.corpus import brown
from collections import Counter

nltk.download('brown')

word_freq = Counter(brown.words())

def word_complexity(word):
    frequency = word_freq[word.lower()]
    return frequency

[nltk_data] Downloading package brown to
[nltk_data]     /Users/phuongdang/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [142]:
word = "pizza"
frequency = word_complexity(word)
print(f"'{word}' appears {frequency} times")

'pizza' appears 3 times


In [115]:
# TESTING
# list of what we care about (nouns, verbs, adjectives)
important = ['JJ', 'JJR', 'JJ', 'NN', 'NNP', 'NNS', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
for pos in pos_tags:
    word = pos[0]
    pos_type = pos[1]
    # pos is something we care about and has no special characters and more than 2 characters (avoid 've and 'll)
    if (pos_type in important) & (word.isalpha()) & (len(word)>2):
        
        # arbitrary complextiy value filter
        if (word_complexity(word)) < 50:
            print(word)
            print(word_complexity(word))
            print('=======================')

sixteen
18
awkward
11
silences
3
dated
19
mom
1


In [200]:
# list of pos tags we care about
important = ['JJ', 'JJR', 'JJ', 'NN', 'NNP', 'NNS', 'RBS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

# Sentence is full string of a tweet, cap is "complexity" value
# Returns one selected word in sentence to mask based on it's part-of-spech tag and how unique the word is
cap = 50
def word_to_mask(sentence, cap):
    tokens = nltk.word_tokenize(sentence)
    pos_tags = nltk.pos_tag(tokens)
    potential_words = []

    for pos in pos_tags:
        word = pos[0]
        pos_type = pos[1]
        # pos is something we care about and has no special characters and more than 2 characters (avoid 've and 'll)
        if (pos_type in important) & (word.isalpha()) & (len(word)>2):
            
            # arbitrary complextiy value filter
            if (word_complexity(word)) < cap:
                potential_words.append(word)

                print(word)
                print(word_complexity(word))
                print('=======================')
                
    random.seed(2003)
    if potential_words == []:
        # increase filter if not found
        return word_to_mask(sentence, cap+5)
    else: 
        choice = random.choice(potential_words)
        return choice

In [159]:
print(funny_df['tweets'][2])
print(word_to_mask(funny_df['tweets'][2]))

[struggling to get out of a hammock] come here and say that
hammock


In [156]:
funny_df

,id,tweets,funny
0,684187,"Sometimes, when I’m on a date with my wife of ...",1
1,509083,Pizza rolls are shaped like little pillows bec...,1
2,413339,[struggling to get out of a hammock] come here...,1
3,880531,If an astronaut goes really fast they’re a fas...,1
4,973171,I toss and turn in bed all night like a beauti...,1
...,...,...,...
138,144630,if you sweat while you eat it should count as ...,1
139,328630,god!! did anyone stop by your house and drop o...,1
140,971535,If you breakdance you buy dance.,1
141,604980,"You can catch a lot of flies with honey, but y...",1


In [193]:
masked_word = []
for i, row in funny_df.iterrows():
    word = word_to_mask(row['tweets'], cap)
    masked_word.append(word)

masked_word

['sixteen',
 'Pizza',
 'struggling',
 'astronaut',
 'toss',
 'Lots',
 'Loan',
 'screenshots',
 'nyc',
 'dating',
 'gon',
 'Mom',
 'neighbor',
 'clever',
 'eating',
 'Bell',
 'didn',
 'canceled',
 'savant',
 'soup',
 'prediction',
 'Mirror',
 'moms',
 'basement',
 'mayor',
 'Flex',
 'lawyer',
 'wives',
 'pole',
 'container',
 'poise',
 'Jehovah',
 'listen',
 'ManBat',
 'Qualifications',
 'songbird',
 'quit',
 'basically',
 'lucky',
 'mom',
 'shaking',
 'pleaser',
 'wasps',
 'villains',
 'roommate',
 'prison',
 'Fellas',
 'rap',
 'combine',
 'olympians',
 'Therapist',
 'Olympic',
 'ostriches',
 'chocolates',
 'Usain',
 'gold',
 'ice',
 'Googling',
 'impressed',
 'don',
 'ankles',
 'Digging',
 'printers',
 'ins',
 'appointment',
 'accounts',
 'cute',
 'banana',
 'staying',
 'shit',
 'depressed',
 'documentary',
 'shut',
 'compete',
 'guy',
 'kinds',
 'bet',
 'Boyfriend',
 'referring',
 'mixes',
 'Diabetes',
 'parent',
 'texts',
 'Nobody',
 'Therapist',
 'puts',
 'killing',
 'funeral',
 'S

In [197]:
# definitely notice overflagging words like 'mom' 'don' and 'gon' 
for i in range(len(masked_word)):
    if len(masked_word[i]) < 4:
        print(i)
        print(masked_word[i])
        print('=============')

8
nyc
10
gon
11
Mom
39
mom
47
rap
56
ice
59
don
63
ins
74
guy
76
bet
100
don
112
PTO
114
Ooh
128
pet
135
gon
137
gon
139
god


In [210]:
funny_df['tweets'][139]

'god!! did anyone stop by your house and drop off your award for having that opinion earlier than most people?'

In [211]:
# manual reviews for now
masked_word[0] = 'mom' # was 'sixteen'
masked_word[8] = 'rat' # was 'nyc'
masked_word[10] = 'seen' # was 'gon'
masked_word[39] = 'soap' # was 'mom'
masked_word[56] = 'profiling' # was 'ice'
masked_word[59] = 'immigrant' # was 'don'
masked_word[114] = 'sun' # was 'Ooh'
masked_word[135] = 'resume' # was 'gon'
masked_word[137] = 'butler' # was 'gon'
masked_word[139] = 'opinion' # was 'god'

In [212]:
len(masked_word)

143

In [213]:
funny_df['masked_word'] = masked_word
funny_df

,id,tweets,funny,masked_word
0,684187,"Sometimes, when I’m on a date with my wife of ...",1,mom
1,509083,Pizza rolls are shaped like little pillows bec...,1,Pizza
2,413339,[struggling to get out of a hammock] come here...,1,struggling
3,880531,If an astronaut goes really fast they’re a fas...,1,astronaut
4,973171,I toss and turn in bed all night like a beauti...,1,toss
...,...,...,...,...
138,144630,if you sweat while you eat it should count as ...,1,sweat
139,328630,god!! did anyone stop by your house and drop o...,1,opinion
140,971535,If you breakdance you buy dance.,1,breakdance
141,604980,"You can catch a lot of flies with honey, but y...",1,catch


In [246]:
import re
def get_mask_index(row):
    tweet = row['tweets'].lower()
    target = row['masked_word'].lower()

    pattern = re.escape(target)
    matches = re.finditer(pattern, tweet)

    if matches:
        indexes = [(match.start(), match.end()) for match in matches]
        return indexes
    else:
        return "Error: target masked word is not in original tweet. "


In [247]:
# GET INDEX TO MASK WORDS
index = []
for i, row in funny_df.iterrows():
    results = get_mask_index(row)
    index.append(results)

funny_df['index'] = index
funny_df

,id,tweets,funny,masked_word,index
0,684187,"Sometimes, when I’m on a date with my wife of ...",1,mom,"[(143, 146)]"
1,509083,Pizza rolls are shaped like little pillows bec...,1,Pizza,"[(0, 5)]"
2,413339,[struggling to get out of a hammock] come here...,1,struggling,"[(1, 11)]"
3,880531,If an astronaut goes really fast they’re a fas...,1,astronaut,"[(6, 15), (44, 53)]"
4,973171,I toss and turn in bed all night like a beauti...,1,toss,"[(2, 6)]"
...,...,...,...,...,...
138,144630,if you sweat while you eat it should count as ...,1,sweat,"[(7, 12)]"
139,328630,god!! did anyone stop by your house and drop o...,1,opinion,"[(76, 83)]"
140,971535,If you breakdance you buy dance.,1,breakdance,"[(7, 17)]"
141,604980,"You can catch a lot of flies with honey, but y...",1,catch,"[(8, 13), (53, 58)]"


In [289]:
def mask_tweet(row):
    tweet = row['tweets']
    new_tweet = tweet

    offset = 0

    for start_i, end_i in row['index']:
        start_i += offset
        end_i += offset

        new_tweet = new_tweet[:start_i] + '[MASK]' + new_tweet[end_i:]

        # the change in string index after [MASK] was added
        # difference between '[MASKED]' and length of masked word 
        offset += len('[MASK]') - (end_i - start_i)

    return new_tweet

In [290]:
masked_tweet = []
for i, row in funny_df.iterrows():
    result = (mask_tweet(row))
    masked_tweet.append(result)
    
funny_df['masked_tweet'] = masked_tweet
funny_df

,id,tweets,funny,masked_word,index,masked_tweet
0,684187,"Sometimes, when I’m on a date with my wife of ...",1,mom,"[(143, 146)]","Sometimes, when I’m on a date with my wife of ..."
1,509083,Pizza rolls are shaped like little pillows bec...,1,Pizza,"[(0, 5)]",[MASK] rolls are shaped like little pillows be...
2,413339,[struggling to get out of a hammock] come here...,1,struggling,"[(1, 11)]",[[MASK] to get out of a hammock] come here and...
3,880531,If an astronaut goes really fast they’re a fas...,1,astronaut,"[(6, 15), (44, 53)]",If an [MASK] goes really fast they’re a f[MASK]
4,973171,I toss and turn in bed all night like a beauti...,1,toss,"[(2, 6)]",I [MASK] and turn in bed all night like a beau...
...,...,...,...,...,...,...
138,144630,if you sweat while you eat it should count as ...,1,sweat,"[(7, 12)]",if you [MASK] while you eat it should count as...
139,328630,god!! did anyone stop by your house and drop o...,1,opinion,"[(76, 83)]",god!! did anyone stop by your house and drop o...
140,971535,If you breakdance you buy dance.,1,breakdance,"[(7, 17)]",If you [MASK] you buy dance.
141,604980,"You can catch a lot of flies with honey, but y...",1,catch,"[(8, 13), (53, 58)]","You can [MASK] a lot of flies with honey, but ..."


In [300]:
funny_df.to_csv('data/masked_tweets.csv', index=False)

In [296]:
print(funny_df['masked_tweet'][4])

I [MASK] and turn in bed all night like a beautiful rotisserie chicken.
